## 生成图像数据文件夹

In [ ]:
import os
os.makedirs("garbageGAN/datasets/data",exist_ok=True)
os.makedirs("garbageGAN/datasets/data/trainA",exist_ok=True)
os.makedirs("garbageGAN/datasets/data/trainB",exist_ok=True)
os.makedirs("yolox-pytorch/VOCdevkit",exist_ok=True)
os.makedirs("yolox-pytorch/VOCdevkit/VOC2007",exist_ok=True)
os.makedirs("yolox-pytorch/VOCdevkit/VOC2007/Annotations",exist_ok=True)
os.makedirs("yolox-pytorch/VOCdevkit/VOC2007/JPEGImages",exist_ok=True)
os.makedirs("yolox-pytorch/VOCdevkit/VOC2007/ImageSets",exist_ok=True)
os.makedirs("yolox-pytorch/VOCdevkit/VOC2007/ImageSets/Main",exist_ok=True)
os.makedirs("yolox-pytorch/test",exist_ok=True)
os.makedirs("yolox-pytorch/test/VOC2007",exist_ok=True)
os.makedirs("yolox-pytorch/test/VOC2007/Annotations",exist_ok=True)
os.makedirs("yolox-pytorch/test/VOC2007/JPEGImages",exist_ok=True)
os.makedirs("yolox-pytorch/test/VOC2007/ImageSets",exist_ok=True)
os.makedirs("yolox-pytorch/test/VOC2007/ImageSets/Main",exist_ok=True)

## 配置环境

In [ ]:
pip install -r requirements.txt

## 处理图像数据集
将图像数据复制进garbagrGAN项目和yolox项目对应的数据位置

In [5]:
import os
import shutil
S_dir="PersonDetect/S"
T_dir="PersonDetect/T"
img_files=os.listdir(os.path.join(S_dir,"imgs"))
xml_files=os.listdir(os.path.join(S_dir,"xml"))
s=0
for i in img_files:
    s=s+1
    name=i.split(".")[0]
    oldpath=os.path.join(S_dir,"imgs",i)  
    newpath=os.path.join("garbageGAN/datasets/PersonDetect/trainA",i)
    if s<=500:shutil.copy(oldpath,newpath)
    
    oldpath=os.path.join(S_dir,"imgs",i)  
    newpath=os.path.join("yolox-pytorch/VOCdevkit/VOC2007/JPEGImages",i)
    shutil.copy(oldpath,newpath)
    
    oldpath=os.path.join(S_dir,"xml",name+".xml")  
    newpath=os.path.join("yolox-pytorch/VOCdevkit/VOC2007/Annotations",name+".xml")
    shutil.copy(oldpath,newpath)

img_files=os.listdir(os.path.join(T_dir,"imgs"))
xml_files=os.listdir(os.path.join(T_dir,"xml"))
for i in img_files:
    name=i.split(".")[0]
    oldpath=os.path.join(T_dir,"imgs",i)  
    newpath=os.path.join("garbageGAN/datasets/PersonDetect/trainB",i)
    shutil.copy(oldpath,newpath)
    
    oldpath=os.path.join(T_dir,"imgs",i)  
    newpath=os.path.join("yolox-pytorch/test/VOC2007/JPEGImages",i)
    shutil.copy(oldpath,newpath)
    
    oldpath=os.path.join(T_dir,"xml",name+".xml")  
    newpath=os.path.join("yolox-pytorch/test/VOC2007/Annotations",name+".xml")
    shutil.copy(oldpath,newpath)

In [2]:
cd garbageGAN

f:\VSCODE\garbageGAN\garbageGAN


## 训练garbageGAN模型

--dataroot 数据集位置，数据集文件结果为dataset/trainA，dataset/trainB，迁移方向为A->B,若要训练其它数据集，需设置同样的文件夹格式

--name 保存各个世代模型文件以及训练日志的位置，后续测试时会选择该文件夹下的latest_net_G.pth作为迁移模型

具体的各个参数的解释在garbageGAN\options文件夹下

需要注意的是，模型训练需要创建一个visdom接口，如果没有开启的话会一直卡在这一步，开启时等候时间可能过长，需要对visdom的库文件进行操作，网上有相关教程

In [11]:
!python train.py --dataroot ./datasets/PersonDetect --name PD_garbageGAN  --model garbageGAN

----------------- Options ---------------
                 CUT_mode: CUT                           
               batch_size: 1                             
                    beta1: 0.5                           
                    beta2: 0.999                         
          checkpoints_dir: ./checkpoints                 
           continue_train: False                         
                crop_size: 256                           
                 dataroot: ./datasets/PersonDetect       	[default: placeholder]
             dataset_mode: unaligned                     
                direction: AtoB                          
              display_env: main                          
             display_freq: 400                           
               display_id: None                          
            display_ncols: 4                             
             display_port: 8097                          
           display_server: http://localhost              
       

Setting up a new session...
Traceback (most recent call last):
  File "d:\py39\lib\site-packages\urllib3\connection.py", line 203, in _new_conn
    sock = connection.create_connection(
  File "d:\py39\lib\site-packages\urllib3\util\connection.py", line 85, in create_connection
    raise err
  File "d:\py39\lib\site-packages\urllib3\util\connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [WinError 10061] 由于目标计算机积极拒绝，无法连接。

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "d:\py39\lib\site-packages\urllib3\connectionpool.py", line 790, in urlopen
    response = self._make_request(
  File "d:\py39\lib\site-packages\urllib3\connectionpool.py", line 496, in _make_request
    conn.request(
  File "d:\py39\lib\site-packages\urllib3\connection.py", line 395, in request
    self.endheaders()
  File "d:\py39\lib\http\client.py", line 1274, in endheaders
    self._send_output(message_body, encod

## 选择迁移模型

经过测试，发现迁移模型在epoch=200的时候迁移图像的效果较好，所以选择该模型文件作为迁移模型

In [ ]:
!cp checkpoints/PD_garbageGAN/200_net_G.pth  checkpoints/PD_garbageGAN/latest_net_G.pth

## 生成迁移后的图像
-- num_test 迁移图像数量，课题里为trainA的所有图像

In [36]:
!python test.py --dataroot ./datasets/PersonDetect --name PD_garbageGAN  --phase train --model garbageGAN --num_test 1144 

----------------- Options ---------------
                 CUT_mode: CUT                           
               batch_size: 1                             
          checkpoints_dir: ./checkpoints                 
                crop_size: 256                           
                 dataroot: ./datasets/PersonDetect       	[default: placeholder]
             dataset_mode: unaligned                     
                direction: AtoB                          
          display_winsize: 256                           
               easy_label: experiment_name               
                    epoch: latest                        
                     eval: False                         
        flip_equivariance: False                         
                  gpu_ids: 0                             
                init_gain: 0.02                          
                init_type: xavier                        
                 input_nc: 3                             
       

In [18]:
cd ../

ϵͳ�Ҳ���ָ����·����


In [ ]:
cd yolox-pytorch

## 生成训练目标检测模型的图像文件索引
若要训练自己的数据集需要在voc_annotation.py里更改class文件位置

In [25]:
!python voc_annotation.py

Generate txt in ImageSets.
train and val size 1029
train size 926
Generate txt in ImageSets done.
Generate 2007_train.txt and 2007_val.txt for train.
Generate 2007_train.txt and 2007_val.txt for train done.
|       worker | 1001 | 
|       person | 1152 | 
|    zhizhipin |  950 | 
| suliaozhipin |  717 | 
|     lvzhipin |  248 | 
|      garbage |   59 | 
|  garbage-bag |  779 | 
|   mianzhipin |   28 | 
|  paomozhipin |  142 | 
|      youtong |   38 | 
|     bumingwu |    2 | 


## 训练目标检测模型

In [ ]:
!python train.py

生成计算评估指标的验证集索引

In [27]:
import os
fp=open(r"test/VOC2007/ImageSets/Main/test.txt","w")
imgdir=r"test/VOC2007/JPEGImages"
files=os.listdir(imgdir)
for i in files:
    name=i.split(".")[0]
    fp.write(name+"\n")
fp.close()

## 计算评估指标
计算迁移前目标检测模型的mAP以及AP指标

需要在yolox-pytorch\yolo.py文件里修改模型文件位置

模型文件储存在yolox-pytorch\logs文件夹里面

计算结果默认保存在yolox-pytorch\map_out文件夹下

In [ ]:
!python get_map.py 

In [ ]:
cd ../

## 处理迁移后图像的脚本

将迁移后的图像加入到目标检测模型训练集中，并生成对应的标注文件

In [20]:
import os 
import shutil
imgdir="yolox-pytorch/VOCdevkit/VOC2007/JPEGImages"
xmldir="yolox-pytorch/VOCdevkit/VOC2007/Annotations"
T_imgdir="garbageGAN/results/PD_garbageGAN/train_latest/images/fake_B"
S_imgdir=r"PersonDetect/S/imgs"
S_xmldir=r"PersonDetect/S/xml"

files=os.listdir(T_imgdir)
for i in files:
    name=i.split(".")[0]
    oldname=os.path.join(T_imgdir,i)
    newname=os.path.join(imgdir,name+"_fake.jpg")
    shutil.copy(oldname,newname)
    oldname=os.path.join(S_xmldir,name+".xml")
    newname=os.path.join(xmldir,name+"_fake.xml")
    shutil.copy(oldname,newname)
    


SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 14-15: truncated \xXX escape (3988472180.py, line 7)

## 再次训练目标检测模型

In [24]:
cd yolox-pytorch

 ������ F �еľ��� �¼Ӿ�
 �������к��� 3280-EA76

 f:\VSCODE\garbageGAN\yolox-pytorch ��Ŀ¼

2024/05/17  19:22    <DIR>          .
2024/05/17  18:08    <DIR>          ..
2024/05/17  18:08    <DIR>          .git
2024/05/11  12:54             1,925 .gitignore
2024/05/17  18:08    <DIR>          .ipynb_checkpoints
2024/05/12  23:21           342,998 2007_train.txt
2024/05/12  23:21            38,219 2007_val.txt
2024/05/17  19:16             8,287 get_map.py
2024/05/17  18:08    <DIR>          img
2024/05/11  12:54            11,347 LICENSE
2024/05/17  19:22    <DIR>          logs
2024/05/17  18:08    <DIR>          model_data
2024/05/17  18:08    <DIR>          nets
2024/05/11  12:54             9,355 predict.py
2024/05/11  12:54            10,792 README.md
2024/05/11  12:54               147 requirements.txt
2024/05/11  12:54             1,368 summary.py
2024/05/17  18:36    <DIR>          test
2024/05/12  23:22            32,516 train.py
2024/05/17  18:08    <DIR>          utils
2024/05/17 

In [ ]:
!python voc_annotation.py 

In [ ]:
!python train.py

## 计算利用迁移后的图像数据集训练的目标检测模型的指标


In [ ]:
!python get_map.py